In [2]:
# Import
import pandas as pd
from finta import TA as ta
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
import sqlalchemy as sql
from sklearn.ensemble import RandomForestRegressor

# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)


In [3]:
ticker1 = "BTC"


ticker2 = "USDT"
pair= "{}{}".format(ticker1,ticker2)
start="2019.1.1"
end = "2022.4.1"
filepath = "Resources/{}_{}_to_{}.csv".format(pair,start,end)

df = pd.read_csv(Path(filepath), index_col= "Date", parse_dates= True, infer_datetime_format = True)
display(df.head())
display(df.tail())

,Open,High,Low,Close,Volume
Date,,,,,
2019-01-01 00:00:00,3701.23,3713.00,3689.88,3700.31,686.367420
2019-01-01 01:00:00,3700.20,3702.73,3684.22,3689.69,613.539115
2019-01-01 02:00:00,3689.67,3695.95,3675.04,3690.00,895.302181
2019-01-01 03:00:00,3690.00,3699.77,3685.78,3693.13,796.714818
2019-01-01 04:00:00,3692.32,3720.00,3685.94,3692.71,1317.452909


,Open,High,Low,Close,Volume
Date,,,,,
2022-03-31 20:00:00,45581.98,45790.00,45519.03,45745.46,1550.11357
2022-03-31 21:00:00,45745.45,45849.99,45647.01,45757.19,887.89890
2022-03-31 22:00:00,45757.19,45820.34,45660.68,45787.85,1058.12591
2022-03-31 23:00:00,45787.85,45787.86,45200.00,45510.34,2889.28168
2022-04-01 00:00:00,45510.35,45644.86,45351.73,45541.50,1838.20071


In [4]:
def appendData(maindf, dataarray, namesarray=None):
    if namesarray==None:
        return maindf.join(pd.DataFrame(dataarray), how='outer')
    return maindf.join(pd.DataFrame(dataarray,columns=namesarray), how='outer')

In [5]:
### Oscillators ###
## RSI
df = appendData(df,ta.RSI(df))
## Sto-%K
df = appendData(df,ta.STOCH(df))
## CCI
df = appendData(df,ta.CCI(df))
## ADX
df = appendData(df,ta.ADX(df))
## DMI (Added to aid in interpreting ADX)
df = appendData(df,ta.DMI(df, 14))
## Awesome
df = appendData(df,ta.AO(df))
## Momentum
df = appendData(df,ta.MOM(df,10))
## MACD (We rename the undescriptive "SIGNAL" here)
df = appendData(df,ta.MACD(df)).rename(columns={"SIGNAL": "MACD SIGNAL"})
## Sto-RSI
df = appendData(df,ta.STOCHRSI(df))
## Williams %R
df = appendData(df,ta.WILLIAMS(df))
## Bull-Bear Power
df = appendData(df,ta.EBBP(df))
## Ultimate (FinTA does not name this column, so we must)
df = appendData(df,ta.UO(df),["UO"])
### Moving Averages ###
sma_ema_averages = [5, 10, 20, 30, 50, 100, 200]
## SMA, EMA
for i in sma_ema_averages:
  df = appendData(df,ta.SMA(df, i))
  df = appendData(df,ta.EMA(df, i))
## VWMA
df = appendData(df, ta.VAMA(df, 20))
## Hull
df = appendData(df,ta.HMA(df, 9))
# Ichimoku -- Base (Kijun) and Conversion (Tenkan) Only
df = appendData(df,ta.ICHIMOKU(df).drop(['senkou_span_a','SENKOU','CHIKOU'], axis=1))

In [6]:
database_connection_string = 'sqlite:///'
engine = sql.create_engine(database_connection_string, echo=True)
sql_statement = """
CREATE TABLE crypto_outcome (
  column_name BLOB,
  _0_precision BLOB,
  _1_precision BLOB,
  _0_recall BLOB,
  _1_recall BLOB,
  _0_f1 BLOB,
  _1_f1 BLOB,
  _0_support BLOB,
  _1_support BLOB
  )
"""
result = engine.execute(sql_statement)

2022-04-10 00:51:32,555 INFO sqlalchemy.engine.Engine 
CREATE TABLE crypto_outcome (
  column_name BLOB,
  _0_precision BLOB,
  _1_precision BLOB,
  _0_recall BLOB,
  _1_recall BLOB,
  _0_f1 BLOB,
  _1_f1 BLOB,
  _0_support BLOB,
  _1_support BLOB
  )

2022-04-10 00:51:32,556 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-10 00:51:32,557 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
models_list = [KNeighborsClassifier(),SVC()]
percent_threshold = .03
investment_amount = 10000
training_length = 12        


for m in models_list:
    model = m
    i = 3
    i_end = 5
    while i <= i_end:
        column_name = str(m) + str(i) + "_period_return"
        df[column_name] = df["Close"].pct_change(periods=i)
        # Initialize the new `Signal` column
        signal_column_name = str(m) + str(i) + "_signal"
        df[signal_column_name] = 0.0
        # Generate signal to buy stock long
        df.loc[(df[column_name] >= (percent_threshold)), signal_column_name] = 1
        # Drop all NaN values from the DataFrame
        df = df.dropna()
        y = df[signal_column_name]
        # set up X for ml 
        X = df[["14 period RSI", "14 period STOCH %K", "20 period CCI", "14 period ADX.", "DI+", "DI-", "AO", "MOM", "MACD", "MACD SIGNAL", "14 period stochastic RSI.", "14 Williams %R", "Bull.", "Bear.", "UO", "5 period SMA", "5 period EMA", "10 period SMA", "10 period EMA", "20 period SMA", "20 period EMA", "30 period SMA", "30 period EMA", "50 period SMA", "50 period EMA", "100 period SMA", "100 period EMA", "200 period SMA", "200 period EMA", "20 period VAMA", "9 period HMA.", "TENKAN", "KIJUN"]].shift().dropna().copy()
        y_count = y.value_counts()
        training_begin = X.index.min()
        # Use the following code to select the ending period for the training data: `training_end = X.index.min() + DateOffset(months=3)`
        training_end = X.index.min() + DateOffset(months=training_length)
        # Generate the X_train and y_train DataFrames using loc to select the rows from `training_begin` up to `training_end`
        # Hint: Use `loc[training_begin:training_end]` for X_train and y_train
        X_train = X.loc[training_begin:training_end]
        y_train = y.loc[training_begin:training_end]

        # Generate the X_test and y_test DataFrames using loc to select from `training_end` to the last row in the DataFrame.
        # Hint: Use `loc[training_end:]` for X_test and y_test
        X_test = X.loc[training_end:]
        y_test = y.loc[training_end:]

        # Use StandardScaler to scale the X_train and X_test data.
        scaler = StandardScaler()
        X_scaler = scaler.fit(X_train)
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)

        # Fit the model to the data using X_train_scaled and y_train
        model = model.fit(X_train_scaled, y_train)

        # Use the trained model to predict the trading signals for the training data.
        training_signal_predictions = model.predict(X_train_scaled)

        # Evaluate the model using a classification report
        training_report_train = classification_report(y_train, training_signal_predictions)
        #print(column_name)
        #print("Training Report")
        #print(training_report_train)

        # Use the trained model to predict the trading signals for the testing data.
        testing_signal_predictions = model.predict(X_test_scaled)

        # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
        #training_report_test = classification_report(y_test, testing_signal_predictions)

        reportdf = pd.DataFrame(classification_report(y_test, testing_signal_predictions, output_dict=True)).transpose()
        engine.execute("INSERT INTO crypto_outcome VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", (column_name, reportdf['precision'][0], reportdf['precision'][1], reportdf['recall'][0], reportdf['recall'][1], reportdf['f1-score'][0], reportdf['f1-score'][1],reportdf['support'][0], reportdf['support'][1]))
       
       
        i+=1
    
sql_query = """
SELECT *
FROM crypto_outcome
"""
outcome = pd.read_sql_query(sql_query, engine, index_col='column_name')
outcome


2022-04-10 00:52:31,329 INFO sqlalchemy.engine.Engine INSERT INTO crypto_outcome VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-04-10 00:52:31,330 INFO sqlalchemy.engine.Engine [raw sql] ('KNeighborsClassifier()3_period_return', 0.9792098714226461, 0.40625, 0.9949952586661047, 0.13948497854077252, 0.9870394564933368, 0.20766773162939295, 18982.0, 466.0)
2022-04-10 00:52:31,330 INFO sqlalchemy.engine.Engine COMMIT
2022-04-10 00:52:34,608 INFO sqlalchemy.engine.Engine INSERT INTO crypto_outcome VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-04-10 00:52:34,609 INFO sqlalchemy.engine.Engine [raw sql] ('KNeighborsClassifier()4_period_return', 0.9798247064660162, 0.2210283960092095, 0.94598190526876, 0.44036697247706424, 0.9626059408085348, 0.29432805314256516, 18790.0, 654.0)
2022-04-10 00:52:34,609 INFO sqlalchemy.engine.Engine COMMIT
2022-04-10 00:52:37,782 INFO sqlalchemy.engine.Engine INSERT INTO crypto_outcome VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-04-10 00:52:37,782 INFO sqlalchemy.engine.Engine [

,_0_precision,_1_precision,_0_recall,_1_recall,_0_f1,_1_f1,_0_support,_1_support
column_name,,,,,,,,
KNeighborsClassifier()3_period_return,0.979184,0.392638,0.994792,0.137339,0.986926,0.203498,19009.0,466.0
KNeighborsClassifier()4_period_return,0.979909,0.221626,0.946059,0.441896,0.962687,0.295199,18817.0,654.0
KNeighborsClassifier()5_period_return,0.983385,0.310721,0.939004,0.634116,0.960682,0.417073,18657.0,809.0
SVC()3_period_return,0.976307,0.833333,0.999947,0.010730,0.987986,0.021186,18997.0,466.0
SVC()4_period_return,0.967526,0.821429,0.999734,0.035168,0.983366,0.067449,18805.0,654.0
SVC()5_period_return,0.960390,0.630769,0.998713,0.050680,0.979177,0.093822,18645.0,809.0
KNeighborsClassifier()3_period_return,0.979210,0.406250,0.994995,0.139485,0.987039,0.207668,18982.0,466.0
KNeighborsClassifier()4_period_return,0.979825,0.221028,0.945982,0.440367,0.962606,0.294328,18790.0,654.0
KNeighborsClassifier()5_period_return,0.983479,0.312044,0.939298,0.635688,0.960881,0.418605,18632.0,807.0
